# Advanced Models

In [ ]:
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow-datasets --quiet
!pip install tensorflow-text --quiet
!pip install -q transformers==4.37.2
#!pip install tfm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import re
import os
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm # Progress Bar
from tqdm.auto import tqdm
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
import tensorflow.keras.backend as K
import tensorflow_text as tf_text
import nltk
from nltk.data import find
from nltk.corpus import stopwords
nltk.download('stopwords')
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Electronics Standard Dataset

In [ ]:
folder_path = '/content/drive/MyDrive/Electronics_data.csv'

In [ ]:
df = pd.read_csv(folder_path)
stripped_df = df.drop(columns=['vote', 'verified', 'reviewTime', 'reviewerID', 'asin','reviewerName'])
stripped_df = stripped_df.dropna()
stripped_df['reviewLength'] = stripped_df['reviewText'].apply(lambda x: len(x.split()))
stripped_df = stripped_df[stripped_df['reviewLength'] > 1]
stripped_df['starNumeral'] = stripped_df['overall'].astype(int)
stripped_df['starCounts'] = stripped_df['starNumeral'] - 1

In [ ]:
X = stripped_df[['reviewText']]
y = stripped_df[['starCounts']]

### CNN - Bidirectional LSTM hybrid model

In [ ]:
def data_split(X, y, test_split = 0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split, shuffle=True)
    #X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size= splits[1] / (splits[0] + splits[1]))
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = data_split(X, y)

In [ ]:
MAX_SEQUENCE_LENGTH = 180
nltk.download('word2vec_sample')

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
wvmodel = KeyedVectors.load_word2vec_format(datapath(word2vec_sample), binary=False)

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


In [ ]:
EMBEDDING_DIM = len(wvmodel['man'])

# initialize embedding matrix
embedding_matrix = np.zeros((len(wvmodel) + 1, EMBEDDING_DIM))
vocab_dict = {}

# build the embedding matrix
for i, word in enumerate(wvmodel.index_to_key):
    embedding_vector = wvmodel[word]

    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        vocab_dict[word] = i

vocab_dict['[UNK]'] = len(vocab_dict)

In [ ]:
X_train_reshaped = tf.squeeze(tf.constant(X_train.values))
X_test_reshaped = tf.squeeze(tf.constant(X_test.values))

In [ ]:
tokenizer = tf_text.WhitespaceTokenizer()
train_tokens = tokenizer.tokenize(X_train_reshaped)
test_tokens = tokenizer.tokenize(X_test_reshaped)

In [ ]:

def docs_to_vocab_ids(tokenized_texts_list):
    """
    converting a list of strings to a list of lists of word ids
    """
    texts_vocab_ids = []
    text_labels = []
    valid_example_list = []
    for i, token_list in tqdm(enumerate(tokenized_texts_list)):

        # Get the vocab id for each token in this doc ([UNK] if not in vocab)
        vocab_ids = []
        for token in list(token_list.numpy()):
            #print(token)
            decoded = token.decode('utf-8', errors='ignore')
            if decoded in vocab_dict:
                vocab_ids.append(vocab_dict[decoded])
            else:
                vocab_ids.append(vocab_dict['[UNK]'])

        vocab_ids = vocab_ids[:MAX_SEQUENCE_LENGTH]
        n_padding = (MAX_SEQUENCE_LENGTH - len(vocab_ids))
        vocab_ids += [vocab_dict['[UNK]']] * n_padding
        texts_vocab_ids.append(vocab_ids)

        #if i % 5000 == 0:
        #    print('Examples processed: ', i)

    print('Total examples: ', i)
    return np.array(texts_vocab_ids)

In [ ]:
train_input_ids = docs_to_vocab_ids(train_tokens)
test_input_ids = docs_to_vocab_ids(test_tokens)

train_input_labels = np.array(y_train)
test_input_labels = np.array(y_test)

0it [00:00, ?it/s]

Total examples:  263167


0it [00:00, ?it/s]

Total examples:  65792


In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

def build_cnn_bilstm_model(learning_rate, dense_layer_dims, dropout_rate, num_filters, kernel_sizes):
  cnn_input_layer = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,))
  cnn_embedding_layer = Embedding(input_dim=len(embedding_matrix),
                                    output_dim=EMBEDDING_DIM,
                                    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                    trainable=False)

  cnn_embeddings = cnn_embedding_layer(cnn_input_layer)

  conv_layers_for_all_kernel_sizes = []
  for kernel_size, filters in zip(kernel_sizes, num_filters):
    conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0015))(cnn_embeddings)
    conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers_for_all_kernel_sizes.append(conv_layer)
  conv_output = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)
  last_hidden_output = keras.layers.Dropout(rate=dropout_rate)(conv_output)
  last_hidden_output = keras.layers.Reshape((1, -1))(last_hidden_output)
  lstm_layer_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, kernel_regularizer=tf.keras.regularizers.l2(0.001)))(last_hidden_output)
  classification_layer = tf.keras.layers.Dense(5, activation='softmax', )(lstm_layer_1)
  opt = Adam(learning_rate=learning_rate)
  model = tf.keras.Model(inputs=[cnn_input_layer], outputs=[classification_layer])
  model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
num_filters = [100, 100, 50, 25]
kernel_sizes = [3, 4, 5, 10]
dense_layer_dims = 50
dropout_rate = 0.3
learning_rate = 0.0001

cnn_lstm_model = build_cnn_bilstm_model(learning_rate, dense_layer_dims, dropout_rate, num_filters, kernel_sizes)
cnn_lstm_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 180)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 180, 300)       │     13,194,600 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 178, 100)       │         90,100 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 177, 100)       │        120,100 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 176, 50)        │         75,050 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_3 (Conv1D)         │ (None, 171, 25)        │         75,025 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d      │ (None, 100)            │              0 │ conv1d[0][0]           │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_1    │ (None, 100)            │              0 │ conv1d_1[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_2    │ (None, 50)             │              0 │ conv1d_2[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_3    │ (None, 25)             │              0 │ conv1d_3[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 275)            │              0 │ global_max_pooling1d[… │
│                           │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 275)            │              0 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 1, 275)         │              0 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 128)            │        174,080 │ reshape[0][0]          │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 5)              │            645 │ bidirectional[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 13,729,600 (52.37 MB)

 Trainable params: 535,000 (2.04 MB)

 Non-trainable params: 13,194,600 (50.33 MB)

In [ ]:
cnn_lstm_model.fit(train_input_ids, train_input_labels, epochs=8, batch_size=48, validation_data=(test_input_ids, test_input_labels), callbacks=[early_stop])

Epoch 1/8
5483/5483 ━━━━━━━━━━━━━━━━━━━━ 50s 8ms/step - accuracy: 0.5573 - loss: 1.5656 - val_accuracy: 0.6347 - val_loss: 1.0739
Epoch 2/8
5483/5483 ━━━━━━━━━━━━━━━━━━━━ 42s 8ms/step - accuracy: 0.6293 - loss: 1.0784 - val_accuracy: 0.6495 - val_loss: 1.0096
Epoch 3/8
5483/5483 ━━━━━━━━━━━━━━━━━━━━ 42s 8ms/step - accuracy: 0.6478 - loss: 1.0178 - val_accuracy: 0.6528 - val_loss: 0.9867
Epoch 4/8
5483/5483 ━━━━━━━━━━━━━━━━━━━━ 42s 8ms/step - accuracy: 0.6556 - loss: 0.9898 - val_accuracy: 0.6631 - val_loss: 0.9626
Epoch 5/8
5483/5483 ━━━━━━━━━━━━━━━━━━━━ 42s 8ms/step - accuracy: 0.6628 - loss: 0.9700 - val_accuracy: 0.6638 - val_loss: 0.9555
Epoch 6/8
5483/5483 ━━━━━━━━━━━━━━━━━━━━ 42s 8ms/step - accuracy: 0.6642 - loss: 0.9590 - val_accuracy: 0.6710 - val_loss: 0.9411
Epoch 7/8
5483/5483 ━━━━━━━━━━━━━━━━━━━━ 42s 8ms/step - accuracy: 0.6679 - loss: 0.9467 - val_accuracy: 0.6743 - val_loss: 0.9387
Epoch 8/8
5483/5483 ━━━━━━━━━━━━━━━━━━━━ 42s 8ms/step - accuracy: 0.6738 - loss: 0.9340 - 

In [ ]:
preds = cnn_lstm_model.predict(test_input_ids)

2057/2057 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


In [ ]:
preds = np.argmax(preds.reshape(-1,5), axis=1)

In [ ]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.63      0.75      0.68     10141
           1       0.39      0.00      0.01      4389
           2       0.34      0.21      0.26      5927
           3       0.47      0.25      0.32     10433
           4       0.74      0.94      0.83     34903

    accuracy                           0.67     65793
   macro avg       0.51      0.43      0.42     65793
weighted avg       0.62      0.67      0.62     65793



### CNN with Multi-Head Attention Model

In [ ]:
def create_attention_CNN_model(learning_rate, dense_layer_dims, dropout_rate, num_filters, kernel_sizes):

  embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix)

  input_layer = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')

  embedding_layer = Embedding(input_dim=len(embedding_matrix),
                                    output_dim=EMBEDDING_DIM,
                                    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                    trainable=False)

  embeddings = embedding_layer(input_layer)

  conv_layers_for_all_kernel_sizes = []
  for kernel_size, filters in zip(kernel_sizes, num_filters):
    conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0008))(embeddings)
    conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers_for_all_kernel_sizes.append(conv_layer)
  conv_output = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)

  num_heads = 4  # Number of attention heads
  attention_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_matrix.shape[1])(embeddings, embeddings)
  attention_output = tf.keras.layers.GlobalAveragePooling1D()(attention_output)

  merged_layer = tf.keras.layers.Concatenate()([conv_output, attention_output])

  hidden_layer = tf.keras.layers.Dense(dense_layer_dims, activation='relu')(merged_layer)
  dropout_layer = tf.keras.layers.Dropout(dropout_rate)(hidden_layer)

  classification_layer = tf.keras.layers.Dense(5, activation='softmax', name='classification')(dropout_layer)

  classification_model = tf.keras.models.Model(inputs=[input_layer], outputs=[classification_layer])
  classification_model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                      metrics=['accuracy'])

  return classification_model

In [ ]:
attention_CNN_model = create_attention_CNN_model(learning_rate, dense_layer_dims, dropout_rate, num_filters, kernel_sizes)
attention_CNN_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 180)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 180, 300)       │     13,194,600 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_4 (Conv1D)         │ (None, 178, 100)       │         90,100 │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_5 (Conv1D)         │ (None, 177, 100)       │        120,100 │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_6 (Conv1D)         │ (None, 176, 50)        │         75,050 │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_7 (Conv1D)         │ (None, 171, 25)        │         75,025 │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_4    │ (None, 100)            │              0 │ conv1d_4[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_5    │ (None, 100)            │              0 │ conv1d_5[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_6    │ (None, 50)             │              0 │ conv1d_6[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_7    │ (None, 25)             │              0 │ conv1d_7[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 180, 300)       │      1,443,900 │ embedding_1[0][0],     │
│ (MultiHeadAttention)      │                        │                │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 275)            │              0 │ global_max_pooling1d_… │
│ (Concatenate)             │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 300)            │              0 │ multi_head_attention[… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 575)            │              0 │ concatenate_1[0][0],   │
│ (Concatenate)             │                        │                │ global_average_poolin… │
├──────────────────────

 Total params: 15,027,830 (57.33 MB)

 Trainable params: 1,833,230 (6.99 MB)

 Non-trainable params: 13,194,600 (50.33 MB)

In [ ]:
attention_CNN_model.fit(train_input_ids, train_input_labels, epochs=8, batch_size=48, validation_data=(test_input_ids, test_input_labels), callbacks=[early_stop])

Epoch 1/8
5483/5483 ━━━━━━━━━━━━━━━━━━━━ 71s 10ms/step - accuracy: 0.6070 - loss: 1.1336 - val_accuracy: 0.6579 - val_loss: 0.9629
Epoch 2/8
5483/5483 ━━━━━━━━━━━━━━━━━━━━ 32s 6ms/step - accuracy: 0.6490 - loss: 0.9886 - val_accuracy: 0.6453 - val_loss: 0.9747


In [ ]:
preds = attention_CNN_model.predict(test_input_ids)
preds = np.argmax(preds.reshape(-1,5), axis=1)
print(classification_report(y_test, preds))

2057/2057 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step
              precision    recall  f1-score   support

           0       0.56      0.77      0.65     10141
           1       0.00      0.00      0.00      4389
           2       0.34      0.17      0.23      5927
           3       0.47      0.20      0.28     10433
           4       0.73      0.93      0.82     34903

    accuracy                           0.66     65793
   macro avg       0.42      0.41      0.39     65793
weighted avg       0.58      0.66      0.60     65793



## Electronics Balanaced Dataset

In [ ]:
folder_path = '/content/drive/MyDrive/Electronics_data_balanced.csv'
balanced_folder_path = '/content/drive/MyDrive/Electronics_data_balanced_test.csv'

In [ ]:
balanced_df = pd.read_csv(folder_path)
X_train = balanced_df[['reviewText']]
y_train = balanced_df[['starCounts']]
balanced_test_df = pd.read_csv(balanced_folder_path)
X_test = balanced_test_df[['reviewText']]
y_test = balanced_test_df[['starCounts']]

In [ ]:
X_train_reshaped = tf.squeeze(tf.constant(X_train.values))
X_test_reshaped = tf.squeeze(tf.constant(X_test.values))

In [ ]:
train_tokens_2 = tokenizer.tokenize(X_train_reshaped)
test_tokens_2 = tokenizer.tokenize(X_test_reshaped)

In [ ]:
train_input_ids_2 = docs_to_vocab_ids(train_tokens_2)
test_input_ids_2 = docs_to_vocab_ids(test_tokens_2)

train_input_labels_2 = np.array(y_train)
test_input_labels_2 = np.array(y_test)

0it [00:00, ?it/s]

Total examples:  381725


0it [00:00, ?it/s]

Total examples:  65792


### CNN - BiLSTM Model

In [ ]:
cnn_lstm_model_2 = build_cnn_bilstm_model(learning_rate, dense_layer_dims, dropout_rate, num_filters, kernel_sizes)
cnn_lstm_model_2.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 180)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 180, 300)       │     13,194,600 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_8 (Conv1D)         │ (None, 178, 100)       │         90,100 │ embedding_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_9 (Conv1D)         │ (None, 177, 100)       │        120,100 │ embedding_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_10 (Conv1D)        │ (None, 176, 50)        │         75,050 │ embedding_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_11 (Conv1D)        │ (None, 171, 25)        │         75,025 │ embedding_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_8    │ (None, 100)            │              0 │ conv1d_8[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_9    │ (None, 100)            │              0 │ conv1d_9[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_10   │ (None, 50)             │              0 │ conv1d_10[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_11   │ (None, 25)             │              0 │ conv1d_11[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_3             │ (None, 275)            │              0 │ global_max_pooling1d_… │
│ (Concatenate)             │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 275)            │              0 │ concatenate_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_1 (Reshape)       │ (None, 1, 275)         │              0 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (None, 128)            │        174,080 │ reshape_1[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 5)              │            645 │ bidirectional_1[0][0]  │
└──────────────────────

 Total params: 13,729,600 (52.37 MB)

 Trainable params: 535,000 (2.04 MB)

 Non-trainable params: 13,194,600 (50.33 MB)

In [ ]:
cnn_lstm_model_2.fit(train_input_ids_2, train_input_labels_2, epochs=8, batch_size=48, validation_data=(test_input_ids_2, test_input_labels_2), callbacks=[early_stop])

Epoch 1/8
7953/7953 ━━━━━━━━━━━━━━━━━━━━ 64s 8ms/step - accuracy: 0.3747 - loss: 1.6939 - val_accuracy: 0.6234 - val_loss: 1.1301
Epoch 2/8
7953/7953 ━━━━━━━━━━━━━━━━━━━━ 61s 8ms/step - accuracy: 0.5046 - loss: 1.2761 - val_accuracy: 0.6115 - val_loss: 1.1188


In [ ]:
preds = cnn_lstm_model_2.predict(test_input_ids_2)

2057/2057 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


In [ ]:
preds = np.argmax(preds.reshape(-1,5), axis=1)

In [ ]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.58      0.73      0.65     10220
           1       0.23      0.16      0.19      4411
           2       0.26      0.33      0.29      6109
           3       0.41      0.28      0.33     10329
           4       0.80      0.81      0.80     34724

    accuracy                           0.62     65793
   macro avg       0.46      0.46      0.45     65793
weighted avg       0.62      0.62      0.62     65793



### CNN - Multihead Attention Model

In [ ]:
cnn_attention_model_2 = create_attention_CNN_model(learning_rate, dense_layer_dims, dropout_rate, num_filters, kernel_sizes)
cnn_attention_model_2.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 180)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 180, 300)       │     13,194,600 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_12 (Conv1D)        │ (None, 178, 100)       │         90,100 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_13 (Conv1D)        │ (None, 177, 100)       │        120,100 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_14 (Conv1D)        │ (None, 176, 50)        │         75,050 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_15 (Conv1D)        │ (None, 171, 25)        │         75,025 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_12   │ (None, 100)            │              0 │ conv1d_12[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_13   │ (None, 100)            │              0 │ conv1d_13[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_14   │ (None, 50)             │              0 │ conv1d_14[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_15   │ (None, 25)             │              0 │ conv1d_15[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 180, 300)       │      1,443,900 │ embedding_3[0][0],     │
│ (MultiHeadAttention)      │                        │                │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_4             │ (None, 275)            │              0 │ global_max_pooling1d_… │
│ (Concatenate)             │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 300)            │              0 │ multi_head_attention_… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_5             │ (None, 575)            │              0 │ concatenate_4[0][0],   │
│ (Concatenate)             │                        │                │ global_average_poolin… │
├──────────────────────

 Total params: 15,027,830 (57.33 MB)

 Trainable params: 1,833,230 (6.99 MB)

 Non-trainable params: 13,194,600 (50.33 MB)

In [ ]:
cnn_attention_model_2.fit(train_input_ids_2, train_input_labels_2, epochs=8, batch_size=48, validation_data=(test_input_ids_2, test_input_labels_2), callbacks=[early_stop])

Epoch 1/8
7953/7953 ━━━━━━━━━━━━━━━━━━━━ 66s 7ms/step - accuracy: 0.4537 - loss: 1.3577 - val_accuracy: 0.6234 - val_loss: 1.0615
Epoch 2/8
7953/7953 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step - accuracy: 0.5202 - loss: 1.2111 - val_accuracy: 0.6311 - val_loss: 1.0132


In [ ]:
preds = cnn_attention_model_2.predict(test_input_ids_2)

2057/2057 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


In [ ]:
preds = np.argmax(preds.reshape(-1,5), axis=1)

In [ ]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.56      0.78      0.65     10220
           1       0.24      0.25      0.25      4411
           2       0.27      0.28      0.28      6109
           3       0.41      0.31      0.35     10329
           4       0.83      0.78      0.80     34724

    accuracy                           0.62     65793
   macro avg       0.46      0.48      0.47     65793
weighted avg       0.63      0.62      0.62     65793



## Clothing, Shoes, and Jewelry Dataset

In [ ]:
folder_path = '/content/drive/MyDrive/Clothing_Shoes_and_Jewelry_data.csv'

In [ ]:
df_3 = pd.read_csv(folder_path)
stripped_df_2 = df_3.drop(columns=['vote', 'verified', 'reviewTime', 'reviewerID', 'asin','reviewerName'])
stripped_df_2 = stripped_df_2.dropna()
stripped_df_2['reviewLength'] = stripped_df_2['reviewText'].apply(lambda x: len(x.split()))
stripped_df_2 = stripped_df_2[stripped_df_2['reviewLength'] > 1]
stripped_df_2['starNumeral'] = stripped_df_2['overall'].astype(int)
stripped_df_2['starCounts'] = stripped_df_2['starNumeral'] - 1

In [ ]:
X = stripped_df_2[['reviewText']]
y = stripped_df_2[['starCounts']]
X_train, X_test, y_train, y_test = data_split(X, y)

In [ ]:
X_train_reshaped = tf.squeeze(tf.constant(X_train.values))
X_test_reshaped = tf.squeeze(tf.constant(X_test.values))

In [ ]:
tokenizer_3 = tf_text.WhitespaceTokenizer()
train_tokens_3 = tokenizer_3.tokenize(X_train_reshaped)
test_tokens_3 = tokenizer_3.tokenize(X_test_reshaped)

### CNN - Bidirectional LSTM Hybrid Model

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
num_filters = [100, 80, 50, 25]
kernel_sizes = [3, 4, 5, 10]
dense_layer_dims = 60
dropout_rate = 0.3
learning_rate = 0.0005

cnn_lstm_model_3 = build_cnn_bilstm_model(learning_rate, dense_layer_dims, dropout_rate, num_filters, kernel_sizes)
cnn_lstm_model_3.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 180)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_4 (Embedding)   │ (None, 180, 300)       │     13,194,600 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_16 (Conv1D)        │ (None, 178, 100)       │         90,100 │ embedding_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_17 (Conv1D)        │ (None, 177, 80)        │         96,080 │ embedding_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_18 (Conv1D)        │ (None, 176, 50)        │         75,050 │ embedding_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_19 (Conv1D)        │ (None, 171, 25)        │         75,025 │ embedding_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_16   │ (None, 100)            │              0 │ conv1d_16[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_17   │ (None, 80)             │              0 │ conv1d_17[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_18   │ (None, 50)             │              0 │ conv1d_18[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_19   │ (None, 25)             │              0 │ conv1d_19[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_6             │ (None, 255)            │              0 │ global_max_pooling1d_… │
│ (Concatenate)             │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_6 (Dropout)       │ (None, 255)            │              0 │ concatenate_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_2 (Reshape)       │ (None, 1, 255)         │              0 │ dropout_6[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_2           │ (None, 128)            │        163,840 │ reshape_2[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 5)              │            645 │ bidirectional_2[0][0]  │
└──────────────────────

 Total params: 13,695,340 (52.24 MB)

 Trainable params: 500,740 (1.91 MB)

 Non-trainable params: 13,194,600 (50.33 MB)

In [ ]:
train_input_ids_3 = docs_to_vocab_ids(train_tokens_3)
test_input_ids_3 = docs_to_vocab_ids(test_tokens_3)

train_input_labels_3 = np.array(y_train)
test_input_labels_3 = np.array(y_test)

0it [00:00, ?it/s]

Total examples:  383642


0it [00:00, ?it/s]

Total examples:  95910


In [ ]:
cnn_lstm_model_3 = build_cnn_bilstm_model(learning_rate, dense_layer_dims, dropout_rate, num_filters, kernel_sizes)
cnn_lstm_model_3.fit(train_input_ids_3, train_input_labels_3, epochs=8, batch_size=48, validation_data=(test_input_ids_3, test_input_labels_3), callbacks=[early_stop])

Epoch 1/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 67s 8ms/step - accuracy: 0.6078 - loss: 1.1762 - val_accuracy: 0.6294 - val_loss: 1.0444
Epoch 2/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 62s 8ms/step - accuracy: 0.6488 - loss: 0.9864 - val_accuracy: 0.6649 - val_loss: 0.9490
Epoch 3/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 62s 8ms/step - accuracy: 0.6531 - loss: 0.9680 - val_accuracy: 0.6623 - val_loss: 0.9287
Epoch 4/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 62s 8ms/step - accuracy: 0.6537 - loss: 0.9621 - val_accuracy: 0.6570 - val_loss: 0.9428
Epoch 5/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 62s 8ms/step - accuracy: 0.6556 - loss: 0.9552 - val_accuracy: 0.6707 - val_loss: 0.9163
Epoch 6/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 62s 8ms/step - accuracy: 0.6558 - loss: 0.9527 - val_accuracy: 0.6677 - val_loss: 0.9185
Epoch 7/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 62s 8ms/step - accuracy: 0.6558 - loss: 0.9513 - val_accuracy: 0.6745 - val_loss: 0.9081
Epoch 8/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 62s 8ms/step - accuracy: 0.6572 - loss: 0.9493 - 

In [ ]:
preds = cnn_lstm_model_3.predict(test_input_ids_3)
preds = np.argmax(preds.reshape(-1,5), axis=1)
print(classification_report(y_test, preds))

2998/2998 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
              precision    recall  f1-score   support

           0       0.52      0.71      0.60      8512
           1       0.39      0.05      0.08      6326
           2       0.39      0.32      0.35     10176
           3       0.51      0.33      0.40     17587
           4       0.77      0.93      0.84     53310

    accuracy                           0.67     95911
   macro avg       0.51      0.47      0.45     95911
weighted avg       0.64      0.67      0.64     95911



### CNN with Multi-head Attention Model

In [ ]:
attention_cnn_model_3 = create_attention_CNN_model(learning_rate, dense_layer_dims, dropout_rate, num_filters, kernel_sizes)
attention_cnn_model_3.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 180)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_6 (Embedding)   │ (None, 180, 300)       │     13,194,600 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_24 (Conv1D)        │ (None, 178, 100)       │         90,100 │ embedding_6[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_25 (Conv1D)        │ (None, 177, 80)        │         96,080 │ embedding_6[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_26 (Conv1D)        │ (None, 176, 50)        │         75,050 │ embedding_6[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_27 (Conv1D)        │ (None, 171, 25)        │         75,025 │ embedding_6[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_24   │ (None, 100)            │              0 │ conv1d_24[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_25   │ (None, 80)             │              0 │ conv1d_25[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_26   │ (None, 50)             │              0 │ conv1d_26[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_27   │ (None, 25)             │              0 │ conv1d_27[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_2    │ (None, 180, 300)       │      1,443,900 │ embedding_6[0][0],     │
│ (MultiHeadAttention)      │                        │                │ embedding_6[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_8             │ (None, 255)            │              0 │ global_max_pooling1d_… │
│ (Concatenate)             │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 300)            │              0 │ multi_head_attention_… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_9             │ (None, 555)            │              0 │ concatenate_8[0][0],   │
│ (Concatenate)             │                        │                │ global_average_poolin… │
├──────────────────────

 Total params: 15,008,420 (57.25 MB)

 Trainable params: 1,813,820 (6.92 MB)

 Non-trainable params: 13,194,600 (50.33 MB)

In [ ]:
attention_cnn_model_3.fit(train_input_ids_3, train_input_labels_3, epochs=8, batch_size=48, validation_data=(test_input_ids_3, test_input_labels_3), callbacks=[early_stop])

Epoch 1/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 86s 10ms/step - accuracy: 0.6256 - loss: 1.0423 - val_accuracy: 0.6677 - val_loss: 0.8939
Epoch 2/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 47s 6ms/step - accuracy: 0.6623 - loss: 0.9071 - val_accuracy: 0.6687 - val_loss: 0.8784
Epoch 3/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 47s 6ms/step - accuracy: 0.6679 - loss: 0.8837 - val_accuracy: 0.6663 - val_loss: 0.8791
Epoch 4/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 47s 6ms/step - accuracy: 0.6694 - loss: 0.8723 - val_accuracy: 0.6735 - val_loss: 0.8537
Epoch 5/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 47s 6ms/step - accuracy: 0.6710 - loss: 0.8648 - val_accuracy: 0.6712 - val_loss: 0.8511
Epoch 6/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 47s 6ms/step - accuracy: 0.6721 - loss: 0.8580 - val_accuracy: 0.6716 - val_loss: 0.8535
Epoch 7/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 47s 6ms/step - accuracy: 0.6735 - loss: 0.8533 - val_accuracy: 0.6737 - val_loss: 0.8459
Epoch 8/8
7993/7993 ━━━━━━━━━━━━━━━━━━━━ 47s 6ms/step - accuracy: 0.6748 - loss: 0.8500 -

In [ ]:
preds = attention_cnn_model_3.predict(test_input_ids_3)
preds = np.argmax(preds.reshape(-1,5), axis=1)
print(classification_report(y_test, preds))

2998/2998 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step
              precision    recall  f1-score   support

           0       0.49      0.74      0.59      8512
           1       0.40      0.05      0.09      6326
           2       0.38      0.39      0.38     10176
           3       0.55      0.25      0.35     17587
           4       0.78      0.93      0.85     53310

    accuracy                           0.67     95911
   macro avg       0.52      0.47      0.45     95911
weighted avg       0.64      0.67      0.63     95911



## Clothing, Shoes, Jewelry Balanced Dataset

In [ ]:
folder_path = '/content/drive/MyDrive/Clothing_data_balanced.csv'
balanced_folder_path = '/content/drive/MyDrive/Clothing_data_balanced_test.csv'

In [ ]:
balanced_df = pd.read_csv(folder_path)
X_train = balanced_df[['reviewText']]
y_train = balanced_df[['starCounts']]
balanced_test_df = pd.read_csv(balanced_folder_path)
X_test = balanced_test_df[['reviewText']]
y_test = balanced_test_df[['starCounts']]

In [ ]:
X_train_reshaped = tf.squeeze(tf.constant(X_train.values))
X_test_reshaped = tf.squeeze(tf.constant(X_test.values))

In [ ]:
train_tokens_4 = tokenizer.tokenize(X_train_reshaped)
test_tokens_4 = tokenizer.tokenize(X_test_reshaped)

In [ ]:
train_input_ids_4 = docs_to_vocab_ids(train_tokens_4)
test_input_ids_4 = docs_to_vocab_ids(test_tokens_4)

train_input_labels_4 = np.array(y_train)
test_input_labels_4 = np.array(y_test)

0it [00:00, ?it/s]

Total examples:  458391


0it [00:00, ?it/s]

Total examples:  95910


### CNN - BiLSTM Model

In [ ]:
cnn_lstm_model_4 = build_cnn_bilstm_model(learning_rate, dense_layer_dims, dropout_rate, num_filters, kernel_sizes)
cnn_lstm_model_4.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 180)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_7 (Embedding)   │ (None, 180, 300)       │     13,194,600 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_28 (Conv1D)        │ (None, 178, 100)       │         90,100 │ embedding_7[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_29 (Conv1D)        │ (None, 177, 80)        │         96,080 │ embedding_7[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_30 (Conv1D)        │ (None, 176, 50)        │         75,050 │ embedding_7[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_31 (Conv1D)        │ (None, 171, 25)        │         75,025 │ embedding_7[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_28   │ (None, 100)            │              0 │ conv1d_28[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_29   │ (None, 80)             │              0 │ conv1d_29[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_30   │ (None, 50)             │              0 │ conv1d_30[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_31   │ (None, 25)             │              0 │ conv1d_31[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_10            │ (None, 255)            │              0 │ global_max_pooling1d_… │
│ (Concatenate)             │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_10 (Dropout)      │ (None, 255)            │              0 │ concatenate_10[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_4 (Reshape)       │ (None, 1, 255)         │              0 │ dropout_10[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_4           │ (None, 128)            │        163,840 │ reshape_4[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 5)              │            645 │ bidirectional_4[0][0]  │
└──────────────────────

 Total params: 13,695,340 (52.24 MB)

 Trainable params: 500,740 (1.91 MB)

 Non-trainable params: 13,194,600 (50.33 MB)

In [ ]:
cnn_lstm_model_4.fit(train_input_ids_4, train_input_labels_4, epochs=8, batch_size=48, validation_data=(test_input_ids_4, test_input_labels_4), callbacks=[early_stop])

Epoch 1/8
9550/9550 ━━━━━━━━━━━━━━━━━━━━ 78s 8ms/step - accuracy: 0.4454 - loss: 1.4273 - val_accuracy: 0.6466 - val_loss: 1.0329
Epoch 2/8
9550/9550 ━━━━━━━━━━━━━━━━━━━━ 73s 8ms/step - accuracy: 0.5111 - loss: 1.2559 - val_accuracy: 0.6497 - val_loss: 1.0213


In [ ]:
preds = cnn_lstm_model.predict(test_input_ids_4)
preds = np.argmax(preds.reshape(-1,5), axis=1)
print(classification_report(y_test, preds))

2998/2998 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
              precision    recall  f1-score   support

           0       0.48      0.69      0.57      8716
           1       0.62      0.00      0.00      6246
           2       0.41      0.16      0.23     10139
           3       0.52      0.19      0.28     17398
           4       0.71      0.96      0.81     53412

    accuracy                           0.65     95911
   macro avg       0.55      0.40      0.38     95911
weighted avg       0.61      0.65      0.58     95911



### CNN with Multi-head Attention Model

In [ ]:
attention_cnn_model_4 = create_attention_CNN_model(learning_rate, dense_layer_dims, dropout_rate, num_filters, kernel_sizes)
attention_cnn_model_4.summary()

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8             │ (None, 180)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_8 (Embedding)   │ (None, 180, 300)       │     13,194,600 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_32 (Conv1D)        │ (None, 178, 100)       │         90,100 │ embedding_8[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_33 (Conv1D)        │ (None, 177, 80)        │         96,080 │ embedding_8[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_34 (Conv1D)        │ (None, 176, 50)        │         75,050 │ embedding_8[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_35 (Conv1D)        │ (None, 171, 25)        │         75,025 │ embedding_8[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_32   │ (None, 100)            │              0 │ conv1d_32[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_33   │ (None, 80)             │              0 │ conv1d_33[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_34   │ (None, 50)             │              0 │ conv1d_34[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_35   │ (None, 25)             │              0 │ conv1d_35[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_3    │ (None, 180, 300)       │      1,443,900 │ embedding_8[0][0],     │
│ (MultiHeadAttention)      │                        │                │ embedding_8[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_11            │ (None, 255)            │              0 │ global_max_pooling1d_… │
│ (Concatenate)             │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 300)            │              0 │ multi_head_attention_… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_12            │ (None, 555)            │              0 │ concatenate_11[0][0],  │
│ (Concatenate)             │                        │                │ global_average_poolin… │
├──────────────────────

 Total params: 15,008,420 (57.25 MB)

 Trainable params: 1,813,820 (6.92 MB)

 Non-trainable params: 13,194,600 (50.33 MB)

In [ ]:
attention_cnn_model_4.fit(train_input_ids_4, train_input_labels_4, epochs=8, batch_size=48, validation_data=(test_input_ids_4, test_input_labels_4), callbacks=[early_stop])

Epoch 1/8
9550/9550 ━━━━━━━━━━━━━━━━━━━━ 77s 7ms/step - accuracy: 0.4646 - loss: 1.3125 - val_accuracy: 0.6382 - val_loss: 0.9756
Epoch 2/8
9550/9550 ━━━━━━━━━━━━━━━━━━━━ 55s 6ms/step - accuracy: 0.5279 - loss: 1.1731 - val_accuracy: 0.6396 - val_loss: 0.9721


In [ ]:
preds = attention_cnn_model_4.predict(test_input_ids_4)
preds = np.argmax(preds.reshape(-1,5), axis=1)
print(classification_report(y_test, preds))

2998/2998 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
              precision    recall  f1-score   support

           0       0.57      0.62      0.59      8716
           1       0.29      0.36      0.32      6246
           2       0.32      0.44      0.37     10139
           3       0.43      0.40      0.42     17398
           4       0.86      0.79      0.82     53412

    accuracy                           0.64     95911
   macro avg       0.50      0.52      0.51     95911
weighted avg       0.66      0.64      0.65     95911



## Testing models on Pet Supplies

In [ ]:
pet_supplies_folder_path = '/content/drive/MyDrive/Pet_Supplies.csv'

In [ ]:
pet_df = pd.read_csv(pet_supplies_folder_path)
pet_df = pet_df.dropna()
pet_df = pet_df.sample(n=20000)
pet_df = df.drop(columns=['vote', 'verified', 'reviewTime', 'reviewerID', 'asin','reviewerName'])
pet_df = pet_df.dropna()
pet_df['reviewLength'] = pet_df['reviewText'].apply(lambda x: len(x.split()))
pet_df = pet_df[pet_df['reviewLength'] > 1]
pet_df['starNumeral'] = pet_df['overall'].astype(int)
pet_df['starCounts'] = pet_df['starNumeral'] - 1
X = pet_df[['reviewText']]
y = pet_df[['starCounts']]


In [ ]:
X_reshaped = tf.squeeze(tf.constant(X.values))

In [ ]:
pet_tokenizer = tf_text.WhitespaceTokenizer()
pet_tokens = pet_tokenizer.tokenize(X_reshaped)

pet_input_ids = docs_to_vocab_ids(pet_tokens)
pet_input_labels = np.array(y)


0it [00:00, ?it/s]

Total examples:  328960


In [ ]:
preds = cnn_lstm_model.predict(pet_input_ids)
preds = np.argmax(preds.reshape(-1,5), axis=1)
print(classification_report(y, preds))

10281/10281 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
              precision    recall  f1-score   support

           0       0.64      0.78      0.71     50960
           1       0.50      0.00      0.01     22154
           2       0.37      0.23      0.28     30107
           3       0.50      0.27      0.35     51874
           4       0.75      0.95      0.84    173866

    accuracy                           0.69    328961
   macro avg       0.55      0.45      0.44    328961
weighted avg       0.64      0.69      0.63    328961



In [ ]:
preds = attention_cnn_model_3.predict(pet_input_ids)
preds = np.argmax(preds.reshape(-1,5), axis=1)
print(classification_report(y, preds))

10281/10281 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step
              precision    recall  f1-score   support

           0       0.57      0.66      0.62     50960
           1       0.39      0.01      0.03     22154
           2       0.31      0.25      0.28     30107
           3       0.49      0.19      0.28     51874
           4       0.71      0.91      0.80    173866

    accuracy                           0.64    328961
   macro avg       0.49      0.41      0.40    328961
weighted avg       0.59      0.64      0.59    328961



## Testing on Automotive Data

In [ ]:
automotive_folder_path = '/content/drive/MyDrive/Automotive_data.csv'

In [ ]:
automotive_df = pd.read_csv(automotive_folder_path)
automotive_df = automotive_df.dropna()
automotive_df = automotive_df.sample(n=20000)
automotive_df = automotive_df.drop(columns=['vote', 'verified', 'reviewTime', 'reviewerID', 'asin','reviewerName'])
automotive_df['reviewLength'] = automotive_df['reviewText'].apply(lambda x: len(x.split()))
automotive_df = automotive_df[automotive_df['reviewLength'] > 1]
automotive_df['starNumeral'] = automotive_df['overall'].astype(int)
automotive_df['starCounts'] = automotive_df['starNumeral'] - 1
X = automotive_df[['reviewText']]
y = automotive_df[['starCounts']]

In [ ]:
y.value_counts()

starCounts
4             12664
3              2827
0              2032
2              1467
1               956
Name: count, dtype: int64

In [ ]:
X_reshaped = tf.squeeze(tf.constant(X.values))

In [ ]:
auto_tokenizer = tf_text.WhitespaceTokenizer()
auto_tokens = pet_tokenizer.tokenize(X_reshaped)

auto_input_ids = docs_to_vocab_ids(auto_tokens)
auto_input_labels = np.array(y)

0it [00:00, ?it/s]

Total examples:  19945


In [ ]:
preds = cnn_lstm_model.predict(auto_input_ids)
preds = np.argmax(preds.reshape(-1,5), axis=1)
print(classification_report(y, preds))

624/624 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0       0.54      0.73      0.62      2032
           1       0.00      0.00      0.00       956
           2       0.40      0.20      0.27      1467
           3       0.51      0.22      0.31      2827
           4       0.79      0.95      0.86     12664

    accuracy                           0.72     19946
   macro avg       0.45      0.42      0.41     19946
weighted avg       0.66      0.72      0.67     19946



In [ ]:
preds = attention_cnn_model_3.predict(auto_input_ids)
preds = np.argmax(preds.reshape(-1,5), axis=1)
print(classification_report(y, preds))

624/624 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step
              precision    recall  f1-score   support

           0       0.41      0.74      0.52      2032
           1       0.28      0.01      0.03       956
           2       0.27      0.29      0.28      1467
           3       0.50      0.21      0.30      2827
           4       0.81      0.86      0.83     12664

    accuracy                           0.67     19946
   macro avg       0.45      0.42      0.39     19946
weighted avg       0.66      0.67      0.65     19946

